# Cortex Cube Quickstart Notebook

Cortex Cube is a multi-agent framework that offers native support for Snowflake tools. Instead of requiring users or developers to choose between RAG with Cortex Search or Text2SQL with Cortex Analyst, let the Cube route the work to the appropriate tool based on the user's request.

CortexCube can be configured to work with 3 types of tools:
- Cortex Search Tool: For unstructured data analysis, which requires a standard RAG access pattern.
- Cortex Analyst Tool: For supporting structured data analysis, which requires a Text2SQL access pattern.
- Python Tool: For supporting custom user operations (using 3rd Party API's), which requires calling arbitrary python.

This notebook walks through how to configure and run a system with all 3 types of tools. The demo is designed to illustrate how the agent can answer questions that require a divserse combination of tools (RAG,Text2SQL, Python, or a combination).

Note that Cortex Cube does not configure the underlying Cortex Search or Cortex Analyst services for the user. Those services must be configured before initializing Cortex Cube.

# Cortex Cube Configuration

## Connection Setup

Authenticate with Snowpark + set token as environment variable for use by the agents.

In [1]:
from CortexCube import CortexCube, CortexSearchTool, CortexAnalystTool, PythonTool
from snowflake.snowpark import Session
import os

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

snowpark = Session.builder.configs(connection_parameters).create()

# Set the Default Token to be used by the agents
os.environ["SNOWFLAKE_API_KEY"] = snowpark.connection.rest.token

DatabaseError: 250001 (08001): Failed to connect to DB: sfpscogs-scs.snowflakecomputing.com:443. Incoming request with IP/Token 108.31.189.75 is not allowed to access Snowflake. Contact your account administrator. For more information about this error, go to https://community.snowflake.com/s/ip-xxxxxxxxxxxx-is-not-allowed-to-access.

## Snowflake Tool Configuration

The Cortex Search Tool and the Cortex Analyst Tool need to be configured as follows. Note that a connection object is required for each config. In the case below we're using the same connection object for both because the services are both in the same account/database/schema. Users have the option to pass in different connection objects as needed.

In [3]:
search_config = {
    "service_name": "SEC_SEARCH_SERVICE",
    "service_topic": "Snowflake's business,product offerings,and performance",
    "data_description": "Snowflake annual reports",
    "retrieval_columns": ["CHUNK"],
    "snowpark_connection": snowpark,
}

analyst_config = {
    "semantic_model": "sp500_semantic_model.yaml",
    "stage": "SEMANTICS",
    "service_topic": "S&P500 company and stock metrics",
    "data_description": "a table with stock and financial metrics about S&P500 companies ",
    "snowpark_connection": snowpark,
}

## Python Tool Configuration

Configuring a Python Tool for Cortex Cube requires 1) Python Callable 2) Tool Description (what does the tool do) 3) Output Description (what does the tool output). 

In the example below we create a NewsTool object that submits a HTTP request to a 3rd Party News API. The python callable is passed into the Python Tool as `news_api_func`.To use the tool below get your free token by signing up for an account at thenewsapi.com or just create your own python function and pass it into the PythonTool.

In [4]:
import requests
import json


class NewsTool:

    def __init__(self, token, limit) -> None:

        self.api_token = token
        self.limit = limit

    def search(self, news_query: str) -> str:

        print("Running News Search tool....")
        news_request = f"""https://api.thenewsapi.com/v1/news/all?api_token={self.api_token}&search={news_query}&language=en&limit={self.limit}"""
        response = requests.get(news_request)
        json_response = json.loads(response.content)["data"]

        return str(json_response)


tool_desc = "searches for relevant news based on user query"
output = "relevant articles"
news_api_func = NewsTool(token=os.getenv("NEWS_API_TOKEN"), limit=3).search

## Cortex Cube Config

After the tools have been configured, initialize them and configure CortexCube

In [5]:
annual_reports = CortexSearchTool(config=search_config, k=5)
sp500 = CortexAnalystTool(config=analyst_config)
news_search = PythonTool(
    python_func=news_api_func, tool_description=tool_desc, output_description=output
)

snowflake_tools = [annual_reports, sp500, news_search]
agent = CortexCube(tools=snowflake_tools)

Cortex Search Tool succesfully initialized
Cortex Analyst Tool succesfully initialized
Python Tool succesfully initialized


# Cortex Cube Testing

The 3 types of questions below are designed to showcase the breadth of tool use patterns possible with Cortex Cube. 

- The Structured Data Questions use the Cortex Analyst agent. 
- The Unstructured Data Questions use either the Cortex Search agent or the Python (News API) agent.
- The last section includes a question that requires the use of both types of tools.

## Structured Data Questions

In [6]:
question = "What is the market cap of Apple? "
agent_answer = await agent.acall(question)
print(agent_answer["output"])

Running Cortex Analyst tool.....
The market cap of Apple is $3,019,131,060,224 or approximately $3.02 trillion.


In [7]:
question = "Which company has the bigger EBITDA, Apple or MSFT? "
agent_answer = await agent.acall(question)
print(agent_answer["output"])

Running Cortex Analyst tool.....
Running Cortex Analyst tool.....
Apple has the bigger EBITDA with $129,629,003,776 compared to Microsoft's $125,981,999,104.


## Unstructured Data Questions

In [8]:
question = "How many customers does Snowflake have?"
agent_answer = await agent.acall(question)
print(agent_answer["output"])

Running Cortex Search tool.....
As of January 31, 2024, Snowflake had 9,437 total customers, increasing from 7,744 customers as of January 31, 2023.


In [9]:
question = "Give me recent news about Azure's AI investments"
agent_answer = await agent.acall(question)
print(agent_answer["output"])

Running News Search tool....


'Recent news about Azure's AI investments includes:\n- Article: Microsoft: Navigating AI Investments Amid Market Volatility Source: [GuruFocus](https://www.gurufocus.com/news/2499558/microsoft-navigating-ai-investments-amid-market-volatility)\n- Article: Microsoft: Leading the AI Revolution Source: [GuruFocus](https://www.gurufocus.com/news/2454844/microsoft-leading-the-ai-revolution)\n- Article: How do the big AI cloud providers stack up at the end of Q2? Bernstein weighs in Source: [Investing.com](https://www.investing.com/news/stock-market-news/how-do-the-big-ai-cloud-providers-stack-up-at-the-end-of-q2-bernstein-weighs-in-432SI-3581725)


## Unstrcutured + Structured Data Questions

In [10]:
question = "What are the top 5 companies in the S&P by EBITDA? Are any of them mentioned as competitors in Snowflake's annual report?"
agent_answer = await agent.acall(question)
print(agent_answer["output"])

Running Cortex Analyst tool.....
Running Cortex Search tool.....
The top 5 companies in the S&P by EBITDA are Apple Inc., Microsoft Corporation, Alphabet Inc., Berkshire Hathaway Inc. New, and Amazon.com, Inc. Among these, Amazon.com, Inc. (through Amazon Web Services), Microsoft Corporation (through Microsoft Azure), and Alphabet Inc. (through Google Cloud Platform) are mentioned as competitors in Snowflake's annual report.


In [2]:
import os
import streamlit as st

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

st.write(connection_parameters)

os.environ["SNOWFLAKE_ACCOUNT"] = connection_parameters["account"]
os.environ["NEWS_API_TOKEN"] = os.getenv("NEWS_API_TOKEN")

2024-10-09 12:13:47.577 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 12:13:47.607 
  command:

    streamlit run /Users/alherrera/Downloads/Cube/cube/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-09 12:13:47.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [6]:
from dotenv import load_dotenv

load_dotenv()
os.getenv("SNOWFLAKE_ACCOUNT")

'sfpscogs-scs'